# ABS Retail Turnover 8501

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Python-set-up" data-toc-modified-id="Python-set-up-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Python set-up</a></span></li><li><span><a href="#Get-data-from-ABS" data-toc-modified-id="Get-data-from-ABS-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Get data from ABS</a></span></li><li><span><a href="#Plot" data-toc-modified-id="Plot-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Plot</a></span></li><li><span><a href="#Finished" data-toc-modified-id="Finished-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Finished</a></span></li></ul></div>

## Python set-up

In [1]:
# system imports
import sys
import pathlib
import calendar

# analytic imports
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

# local imports
from finalise_plot import finalise_plot
from abs_common import (
    get_fs_constants,
    get_plot_constants,
    get_ABS_meta_and_data,
    get_identifier,
    recalibrate_series,
)

# pandas display settings
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

# plotting stuff
plt.style.use('fivethirtyeight')

In [2]:
catalogue_id = '8501'
source, CHART_DIR, META_DATA = get_fs_constants(catalogue_id)

## Get data from ABS

In [3]:
# do the data capture and extraction
abs_dict = get_ABS_meta_and_data(catalogue_id)
if abs_dict is None:
    sys.exit(-1)

Retrieving zip-file from cache ./ABS_CACHE/-statistics-industry-retail-and-wholesale-trade-retail-trade-australia-jul-2022-All-Time-Series-Spreadsheets.zip


## Plot

In [4]:
meta = abs_dict[META_DATA]
RECENT, plot_times, plot_tags = get_plot_constants(meta)

In [5]:
table = '1'
df = abs_dict[table]
plots = meta[meta['Table'] == table]['Data Item Description'].unique()

series_type = "Seasonally Adjusted"

for plot in plots:
    id, units = get_identifier(meta, plot,
                           series_type, table)
    series, units = recalibrate_series(df[id], units)
    
    for start, tag in zip(plot_times, plot_tags):
        plot_data = series[series.index > start] if start else series
        ax = plot_data.plot(lw=2)
        title = f"Retail Trade: {plot}".replace(" ;  Total (State) ; ", "")
        title = title.replace(" ;", "").replace(' retailing', '').replace(' services', '')
        finalise_plot(ax, title, 
                      f'{units} (for month)', tag, CHART_DIR,
                      rfooter=f'{source}s {table}',
                      lfooter=f'From {series_type.lower()} series. '
                               'Turnover at current prices. ',
                      #show=True,
                     )

## Finished

In [6]:
print('Finished')

Finished
